In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.output_result { max-width:90% !important; }</style>"))

In [2]:
import json 
from pprint import pprint 
import pandas as pd
import numpy as np
from pathlib import Path
import os
import re
from tqdm.notebook import trange, tqdm

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str or pathlib.Path): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} JSON files in '{dirpath}'.")

In [5]:
data_path = Path("ALGORITM/")
walk_through_dir(data_path)
files = os.listdir('ALGORITM/')

There are 0 directories and 627 JSON files in 'ALGORITM'.


In [6]:
# Get all file paths - все файлы в папке ALGORITM
file_path_list = list(data_path.glob("*.json"))

i = 0
file_id=list()
print(type(file_id))
for file in file_path_list: 
   
    #print(f"{i+1} file path: {file_path_list[i]}")
    fp = str(file_path_list[i])
    file_id.append(fp[-10:-5])
    i+=1

<class 'list'>


In [7]:
#k = 0
load_data = pd.DataFrame()
ld = pd.DataFrame()
print(len(files))
for f in trange(len(files)): 
    df = pd.read_json('ALGORITM/'+ files[f])
    file_id = files[f][-10:-5]
    length = len(df[df.index =='boxes'].values[0][1])
    print(length)
    boxes = pd.DataFrame()
    all_cargoes_space_result = pd.DataFrame()
 
    data0 = df[df.index =='cargo_space'].values[0][1]
    atom_df = {
    'loading_size': data0['loading_size'],
    'calculation_info': data0['calculation_info']
     }

    temp = pd.json_normalize(atom_df)
    all_cargoes_space_result = pd.concat([all_cargoes_space_result, temp], axis=0).reset_index(drop=True)


    # вывод таблиц с размерами и свойствами коробок, а их дофига...
    for i in trange(length):
        data = df[df.index =='boxes'].values[0][1][i]
        #print(length)
        #stacking = data['stacking'].replace({True: 1, False: 0})
        #turnover = data['turnover'].replace({True: 1, False: 0})                                  
        atom_df = pd.DataFrame({
                   'width': data['size']['width'],
                   'height': data['size']['height'],
                   'length': data['size']['length'],
                   'box_volume': data['size']['width']*data['size']['height']*data['size']['length'],
                   'stacking': data['stacking'],
                   'turnover': data['turnover']}, index=[0])
        atom_df.stacking = atom_df.stacking.replace({True: 1, False: 0})
        atom_df.turnover = atom_df.turnover.replace({True: 1, False: 0})
        boxes = pd.concat([boxes,atom_df], axis=0).reset_index(drop=True)

    load_data = [{'load_id':file_id,'density_percent':all_cargoes_space_result['calculation_info.density_percent'][0], 
         'filling_space_percent':all_cargoes_space_result['calculation_info.filling_space_percent'][0],    # target метрика
         'loading_width':all_cargoes_space_result['loading_size.width'][0],     # размер контейнера
         'loading_height':all_cargoes_space_result['loading_size.height'][0],   # размер контейнера
         'loading_length':all_cargoes_space_result['loading_size.length'][0],   # размер контейнера
         'boxes_volume':boxes['box_volume'].sum(),        # суммарный объем коробок в контнйнере (при нормализации по размерам контейнера практически идентичен target метрике)
         'r_width':boxes['width'].min() / boxes['width'].max(),      # отношение минимальной ширины коробки к максимальной
         'r_height':boxes['height'].min() / boxes['height'].max(),   # отношение минимальной высоты коробки к максимальной
         'r_length':boxes['length'].min() / boxes['length'].max(),   # отношение минимальной длины коробки к максимальной
         #'r_loading_width':boxes['width'].max() / all_cargoes_space_result['loading_size.width'][0],      # отношение ширины контейнера к максимальной ширине коробки
         #'r_loading_height':boxes['height'].max() / all_cargoes_space_result['loading_size.height'][0],   # отношение высоты контейнера к максимальной высоте коробки
         #'r_loading_length':boxes['length'].max() / all_cargoes_space_result['loading_size.length'][0],   # отношение длины контейнера к максимальной длине коробки
         'mean_width':boxes['width'].mean(),    # средняя ширина коробки в контейнере
         'mean_height':boxes['height'].mean(),  # средняя высота коробки в контейнере
         'mean_length':boxes['length'].mean(),  # средняя длина коробки в контейнере
         'mean_stacking':boxes['stacking'].mean(),   # усредненная штабелируемость 
         'mean_turnover':boxes['turnover'].mean(),   # усредненная кантуемость 
         'boxes_qty':length}]        # количество коробок в контейнере


        # создание таблицы свойств карго со всех файлов  


    ld = pd.concat((ld, pd.DataFrame(load_data)))


627


  0%|          | 0/627 [00:00<?, ?it/s]

270


  0%|          | 0/270 [00:00<?, ?it/s]

33


...

  0%|          | 0/1 [00:00<?, ?it/s]

2


  0%|          | 0/2 [00:00<?, ?it/s]

10


  0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
 # вывод таблицы свойств карго со всех файлов 
ld.reset_index(drop=True)

,load_id,density_percent,filling_space_percent,loading_width,loading_height,loading_length,boxes_volume,r_width,r_height,r_length,mean_width,mean_height,mean_length,mean_stacking,mean_turnover,boxes_qty
0,11834,81.76416,81.72327,800.0,2000.0,1200.0,1.569087e+09,0.191919,0.369775,0.354271,130.962963,175.966667,214.596296,1.000000,1.000000,270
1,12729,82.50000,51.56250,800.0,2000.0,1200.0,9.900000e+08,1.000000,1.000000,1.000000,300.000000,250.000000,400.000000,1.000000,0.000000,33
2,13579,94.44444,64.24381,2500.0,2100.0,7500.0,2.529600e+10,1.000000,1.000000,1.000000,800.000000,1550.000000,1200.000000,0.000000,0.000000,17
3,13580,74.02778,67.29798,800.0,1650.0,1200.0,1.066000e+09,0.666667,0.666667,0.600000,278.787879,269.696970,415.151515,1.000000,1.000000,33
4,14208,100.00000,58.26317,2490.0,2590.0,13490.0,5.068800e+10,1.000000,1.000000,1.000000,800.000000,1600.000000,1200.000000,0.000000,0.000000,33
5,14457,92.20825,90.27382,2480.0,2860.0,13600.0,8.708000e+10,1.000000,0.520000,0.733333,175.000000,209.751553,2462.939959,1.000000,0.000000,966
6,14770,55.72132,44.63278,2400.0,2500.0,6100.0,1.633560e+10,0.390000,0.375000,0.557554,1605.714286,1597.142857,1040.000000,0.428571,0.000000,7
7,14771,82.70950,72.29493,2490.0,2590.0,13490.0,6.289540e+10,0.437500,0.198675,0.503676,843.333333,819.074074,1584.814815,1.000000,0.000000,54
8,14772,58.69138,47.83901,2450.0,2650.0,8800.0,2.733234e+10,0.310000,0.375000,0.557554,1280.000000,1327.777778,912.222222,0.777778,0.111111,18
9,14775,55.72132,44.63278,2400.0,2500.0,6100.0,1.633560e+10,0.390000,0.375000,0.557554,1605.714286,1597.142857,1040.000000,0.428571,0.000000,7


In [15]:
ld.to_csv('loading_data.csv', index=False)

In [16]:
# таблица с нормализованными данными (по размерам контейнера)

lsd = ld.copy()
lsd['mean_width_norm'] = ld['mean_width'] / ld['loading_width']
lsd['mean_height_norm'] = ld['mean_height'] / ld['loading_height']
lsd['mean_length_norm'] = ld['mean_length'] / ld['loading_length']
#lsd['bv_norm'] = ld['boxes_volume'] / ld['loading_width'] / ld['loading_height'] / ld['loading_length']

In [17]:
lsd = lsd.drop(['load_id', 'density_percent', 'loading_width','loading_height','loading_length', 'mean_width','mean_height','mean_length','mean_stacking', 'mean_turnover', 'boxes_volume'], axis=1)

In [18]:
lsd

,filling_space_percent,r_width,r_height,r_length,boxes_qty,mean_width_norm,mean_height_norm,mean_length_norm
0,81.72327,0.191919,0.369775,0.354271,270,0.163704,0.087983,0.178830
0,51.56250,1.000000,1.000000,1.000000,33,0.375000,0.125000,0.333333
0,64.24381,1.000000,1.000000,1.000000,17,0.320000,0.738095,0.160000
0,67.29798,0.666667,0.666667,0.600000,33,0.348485,0.163453,0.345960
0,58.26317,1.000000,1.000000,1.000000,33,0.321285,0.617761,0.088955
0,90.27382,1.000000,0.520000,0.733333,966,0.070565,0.073340,0.181099
0,44.63278,0.390000,0.375000,0.557554,7,0.669048,0.638857,0.170492
0,72.29493,0.437500,0.198675,0.503676,54,0.338688,0.316245,0.117481
0,47.83901,0.310000,0.375000,0.557554,18,0.522449,0.501048,0.103662
0,44.63278,0.390000,0.375000,0.557554,7,0.669048,0.638857,0.170492


In [19]:
lsd.to_csv('loading_norm_data.csv', index=False)